In [1]:
import numpy as np
from fluid import Fluid
import scipy as sp
from scipy import optimize as op
from scipy.optimize import minimize

import json

import warnings
warnings.filterwarnings("ignore")

In [2]:
#calculating specific speed for sizing pump and turbine
co2 = Fluid('co2')

m_dot_c = 15
P_pump_discharge = 20e6

#Givens
m_dot_s = 3.2 # mass flow of the continuous supply being sequestered (s for sequestration)
#State 5 is the incoming sequestration state 
T5 = 273.15 + 20 #assuming this is the state going into sequestration (at the beginning of pipe)
P5 = 6e6

#State 1
m_dot_1 = m_dot_s + m_dot_c
T1 = T5 # to match sequestration state
P1 = P5 #to match sequestration state

co2 = Fluid('co2')
s1 = co2.entropy(T=T1, P=P1)
h1 = co2.enthalpy(T = T1, P = P1)

#State 2
m_dot_2 = m_dot_s + m_dot_c
s2 = s1 #Pump modeled as isentropic
P2 = P_pump_discharge #Free variable

h2 = co2.enthalpy(S = s2, P = P2)
T2 = co2.temperature(S = s2, P = P2)

pow_in = m_dot_2 * (h2 - h1)

#State 3
dP=88925176.31 #pipe_dP #for now
dT=141.27826#pipe_dT #for now
m_dot_3 = m_dot_c
P3 = 19.35e6 #dP is the pressure loss of the underground system
T3 = 125+273.15 #dT is the temperature increase after the underground system

s3 = co2.entropy(T=T3, P=P3)
h3 = co2.enthalpy(T = T3, P = P3)

#State 4
m_dot_4 = m_dot_c
s4 = s3 #Turbine modeled as isentropic
P4 = P1 # Condenser modeled as isobarric

h4 = co2.enthalpy(S = s4, P = P4)
pow_out = m_dot_4 * (h3 - h4)

net_pow = (pow_out - pow_in)

omega_t =  1800 * 60 / (2*np.pi) #rotational speed, adjust to get a reasonable efficiency
print(T3, P3)
print(co2.density(T = T3, P = P3))
Q_t = m_dot_c / co2.density(T = T3, P = P3) #volumetric flow rate
head_change_t = P3 - P4 / (co2.density(T = T3, P = P3) * 9.81)

specific_speed_d_t = omega_t * Q_t**.5 / (head_change_t*9.81)**.75

print(specific_speed_d_t * 129)

omega_p = 1800 * 60 / (2*np.pi) #rotational speed, adjust to get a reasonable efficiency
Q_p = (m_dot_c + m_dot_s) / co2.density(T = T1, P = P1)#volumetric flow rate
head_change_p = P2 - P1 / (co2.density(T = T1, P = P1) * 9.81)

specific_speed_d_p = omega_p * Q_p**.5 / (head_change_p*9.81)**.75

print(specific_speed_d_p * 129)

D_s_t = 10 #for efficiency of .7
D_t = D_s_t * Q_t**.5 / head_change_t**.25

print(D_t)

D_s_p = 10 #for efficiency of .7
D_p = D_s_p * Q_p**.5 / head_change_p**.25

print(D_p)

398.15 19350000.0
371.18895353540177
0.27564299522942776
0.2039738305828129
0.0303100820106192
0.022803374667855727
